<a href="https://colab.research.google.com/github/cjdolan/HighResolutionSemanticClassification/blob/main/ATML_Build_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook for automatically building a satellite imagery dataset for semantic classification and object detection.

Author: Connor Dolan

In [ ]:
!pip install xmltodict
!pip install geopy
!pip install d2l
!pip install wget
!pip install utm
!pip install rasterio
!pip install swifter

In [ ]:
USERNAME = "cjdolan"
PASSWORD = "1998B@seb@!!29"

In [ ]:
import pandas as pd
import requests
from collections import defaultdict
import time
import json
import xml
import xml.etree.ElementTree as ET
import xmltodict
from tqdm import tqdm
from urllib.parse import urljoin
from shapely.geometry import Polygon
import numpy as np
import wget
from multiprocessing.pool import ThreadPool
import zipfile
from glob import glob
import swifter
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
import os
import shutil

auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!mkdir bucket_data
!gcsfuse --implicit-dirs atml_bucket bucket_data
headers = {}
payload = {}

In [ ]:
print(len(np.unique(glob('/content/bucket_data/*.tif'))))

In [ ]:
%%time
r = requests.get('https://overpass-api.de/api/interpreter?data=way[name=%22Fulton%20County%20Airport-Brown%20Field%22];(._;%3E;);way(around:40000)[aeroway=%22aerodrome%22];(._;%3E;);out%20center;')
xmltodict.parse(r.text)['osm']['way'][0]['center']

In [ ]:
xmltodict.parse(r.text)['osm']

In [ ]:
def get_name(tags):
    if isinstance(tags, list):
        for t in tags:
            if t['@k'] == 'name':
                return t['@v']

In [ ]:
def get_node(nodes, way_id):
  for n in nodes:
    if way_id == n['@id']:
      return n

def get_bb(nodes, ways):
  left = right = top = bottom = None
  for w in ways:
    node = get_node(nodes, w['@ref'])
    lat, lon = node['@lat'], node['@lon']
    if left == None:
      left = right = lon
      top = bottom = lat
    else:
      if lon < left:
        left = lon
      if lon > right:
        right = lon
      if lat < bottom:
        bottom = lat
      if lat > top:
        top = lat
  return left, right, top, bottom

#get_bb(xmltodict.parse(r.text)['osm']['node'], xmltodict.parse(r.text)['osm']['way'][0]['nd'])

In [ ]:
df = pd.DataFrame(columns=['Name', 'Lat', 'Long', 'Class', 'Left', 'Right', 'Top', 'Bottom'])

In [ ]:
%%time
def default_value():
    return -1



def get_addresses_of_poi(classes, tags, num_samples, position):
    result_total = []
    for i, t in enumerate(tags):
        poi_result = []
        seen = defaultdict(default_value)
        load_bar = tqdm(total=num_samples[i], position=position, leave=True)
        stack = []

        r = 40000
        n = 'Fulton County Airport-Brown Field'.replace(' ', '%20')
        k = t[0]
        v = t[1]
        query = 'https://overpass-api.de/api/interpreter?data=way(33.76,-84.52,33.78,-84.57)[name=%22{name}%22];(._;%3E;);way(around:{radius})[{key}=%22{value}%22];(._;%3E;);out%20center;'.format(key=k, value=v, name=n, radius=r)
        result = requests.get(query)
        while 'osm' not in xmltodict.parse(result.text).keys():
          time.sleep(2)
          result = requests.get(query)
        nodes = xmltodict.parse(result.text)['osm']['node']
        result = xmltodict.parse(result.text)['osm']['way']
        left, right, top, bottom = get_bb(nodes, result[0]['nd'])
        stack.append([get_name(result[0]['tag']), result[0]['center']['@lat'], result[0]['center']['@lon'], classes[i],
                     left, right, top, bottom])
        while stack:
            cur = stack[0]
            del stack[0]
            if seen[cur[0]] < 0:
                seen[cur[0]] = 1
                poi_result.append(cur)
                load_bar.update(1)
                center_point = (cur[1], cur[2])
                if cur[0] != None:
                    try:
                        n = cur[0].replace(' ', '%20')
                        query = 'https://overpass-api.de/api/interpreter?data=way\
                                      [name=%22{name}%22];\
                                      (._;%3E;);way\
                                      (around:{radius})[{key}=%22{value}%22];\
                                      (._;%3E;);out%20center;'.format(key=k, value=v, name=n, radius=r)
                        try:
                          result = requests.get(query)
                          nodes = xmltodict.parse(result.text)['osm']['node']
                          result = xmltodict.parse(result.text)['osm']['way']
                        except:
                          time.sleep(1)
                          result = requests.get(query)
                          nodes = xmltodict.parse(result.text)['osm']['node']
                          result = xmltodict.parse(result.text)['osm']['way']
                        for way in result:
                            if isinstance(way, dict):
                                if seen[get_name(way['tag'])] < 0:
                                  left, right, top, bottom = get_bb(nodes, way['nd'])
                                  stack.append([get_name(way['tag']), way['center']['@lat'], way['center']['@lon'], classes[i],
                                              left, right, top, bottom])
                    except:
                        pass
            load_bar.set_description('POI: %s   Number of samples: %i' % (classes[i], len(poi_result)))
            if len(poi_result) > num_samples[i]:
                break

        result_total += poi_result

    return result_total

final_results = []
classes = ['power_plant', 'stadium','airport']
osm_names = [['power','plant'], ['leisure', 'stadium'], ['aeroway', 'aerodrome']]
# pool = ThreadPool(10)
# for i in range(len(classes)):
#   final_results.append(pool.apply_async(get_addresses_of_poi, args=([classes[i]], [osm_names[i]], [1500], i)))
# pool.close()
# pool.join()
# final_results = [r.get() for r in final_results]
r = get_addresses_of_poi(classes, osm_names, [1500, 1500, 1500], 0)
result = []
for c in final_results:
  for elem in c:
    result.append(elem)
new_df = pd.DataFrame(r, columns=['Name', 'Lat', 'Long', 'Class', 'Left', 'Right', 'Top', 'Bottom'])
df = pd.concat([df, new_df])

In [ ]:
df

In [ ]:
df.to_csv('/content/drive/MyDrive/Temp_DF_Checkpoint.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Temp_DF_Checkpoint.csv')

In [ ]:
API_URL = "https://m2m.cr.usgs.gov/api/api/json/stable/"
url = API_URL
session = requests.Session()

def login(username, password):
  """Get an API key.
  Parameters
  ----------
  username : str
      EarthExplorer username.
  password : str
      EarthExplorer password.
  """
  login_url = urljoin(url, "login")
  payload = {"username": username, "password": password}
  r = session.post(login_url, json.dumps(payload))
  session.headers["X-Auth-Token"] = r.json().get("data")

def get_resolution(metadata):
  for m in metadata:
    if m['fieldName'] == 'Resolution':
      return m['value']

def scene_search(lat1, lon1, lat2, lon2, datasetName='NAIP', maxResults=5):
  search_url = urljoin(url, "scene-search")
  spatialFilter = {
      'filterType': 'mbr',
      'lowerLeft': {
          'latitude':lat1,
          'longitude':lon1
      },
      'upperRight': {
          'latitude':lat2,
          'longitude':lon2
      }
  }
  cloudCoverFilter = {'min':0, 'max':5}
  sceneFilter = {'spatialFilter':spatialFilter, 'cloudCoverFilter':cloudCoverFilter}
  search_request = {'datasetName': datasetName, 'maxResults': maxResults, 'sceneFilter':sceneFilter, 'metadataType':'full'}
  r = session.post(search_url, json.dumps(search_request)).json()
  for result in r['data']['results']:
    res = get_resolution(result['metadata'])
    if float(res) == 1.0:
      return result['entityId'], result['spatialCoverage']['coordinates'][0]


def get_product_id(entityId):
  search_url = urljoin(url, "download-options")
  payload = {'datasetName':'NAIP', 'entityIds':[entityId]}
  r = session.post(search_url, json.dumps(payload)).json()
  return r['data'][0]['id'], r['data'][0]['displayId']


def download_scene(entityId, productId):
  dl_req_url = urljoin(url, "download-request")
  download_request = {
      'downloads': [{
        'entityId': entityId,
        'productId': productId
      }]
  }
  r = session.post(dl_req_url, json.dumps(download_request)).json()
  return r

def bulk_download_scene(entityId, productId):
  dl_req_url = urljoin(url, "download-request")
  downloads = []
  for i, e in enumerate(entityId):
    downloads.append({'entityId':e, 'productId':productId[i]})
  download_request = {
      'downloads': downloads
  }
  r = session.post(dl_req_url, json.dumps(download_request)).json()
  return r

def download_retrieve():
  dl_req_url = urljoin(url, "download-retrieve")
  download_request = {
  }
  r = session.post(dl_req_url, json.dumps(download_request)).json()
  return r

def remove_download(downloadId):
  dl_req_url = urljoin(url, "download-remove")
  downloads = []
  download_request = {
      'downloadId': downloadId
  }
  r = session.post(dl_req_url, json.dumps(download_request)).json()
  return r

def build_bb(points):
  return [(points[1], points[0]), (points[1], points[2]), (points[3], points[0]), (points[3], points[2])]

def overlap2(rect1,rect2):
    p1 = Polygon([rect1[0], rect1[1],rect1[2],rect1[3]])
    p2 = Polygon([rect2[0], rect2[1],rect2[2],rect2[3]])
    return(p1.intersects(p2))
rect1 = [[-84.6300721, 33.9973916], [-84.5593611, 33.9960083], [-84.5573805, 34.0651027], [-84.6281499, 34.0664888]]
rect2 = build_bb([34.0106131, -84.6085182, 34.0181462, -84.5835407])
print(overlap2(rect1,rect2))

In [ ]:
import json
 
# Data to be written
dictionary = {
    "username": USERNAME,
    "password": PASSWORD
}
 
with open("data.json", "w") as outfile:
    json.dump(dictionary, outfile)

In [ ]:
read_df = False
if read_df:
  df = pd.read_csv('/content/drive/MyDrive/GeoData_large.csv')

In [ ]:
df

In [ ]:
%%time
import json
with open('data.json', 'r') as openfile:
  json_file = json.load(openfile)
  username, password = json_file['username'], json_file['password']
  login(username, password)
  df['entity_id'] = ''
  df['product_id'] = ''
  for index, row in tqdm(df.iterrows(), total=df.shape[0], leave=True, position=0):
    if row.entity_id == '':
      try:
        entityId, scene_box = scene_search(float(row.Bottom), float(row.Right), float(row.Top), float(row.Left))
        product_id, file_name = get_product_id(entityId)
        df.at[index, 'entity_id'] = entityId
        df.at[index, 'product_id'] = product_id
        df.at[index, 'fileName'] = file_name
        temp = df[df.entity_id == '']
        for t_index, t_row in temp.iterrows():
          box = build_bb([t_row.Bottom, t_row.Right, t_row.Top, t_row.Left])
          if overlap2(scene_box, box):
            df.at[index, 'entity_id'] = entityId
            df.at[index, 'product_id'] = product_id
            df.at[index, 'fileName'] = file_name
      except:
        pass

In [ ]:
df.to_csv('/content/drive/MyDrive/Temp_Geo_data_new.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Temp_Geo_data_new.csv')

In [ ]:
# %%time
# e_ids, p_ids = df[~pd.isna(df.fileName)].iloc[0:10]['entity_id'].tolist(), df[~pd.isna(df.fileName)].iloc[0:10]['product_id'].tolist()
# with open('data.json', 'r') as openfile:
#   json_file = json.load(openfile)
#   username, password = json_file['username'], json_file['password']
#   login(username, password)
#   r = bulk_download_scene(e_ids, p_ids)
#   while len(r['data']['preparingDownloads']) > 0:
#     r = bulk_download_scene(e_ids, p_ids)
#     print(r)

In [ ]:
def get_bounding_box(x_dim, y_dim, lat_min, lon_min, lat_max, lon_max):
  return x_dim*lon_min, x_dim*lon_max, y_dim*lat_min, y_dim*lat_max

In [ ]:
df = df[~pd.isna(df.fileName)]

In [ ]:
df.shape

In [ ]:
import utm
from osgeo import gdal
def get_utm(lon, lat, imgfile, x_or_y):
  try:
    u = utm.from_latlon(float(lat), float(lon))

    driver = gdal.GetDriverByName('GTiff')

    filename = '/content/data/' + str(imgfile).lower() + '.tif' #path to raster

    dataset = gdal.Open(filename)
    band = dataset.GetRasterBand(1)

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    transform = dataset.GetGeoTransform()

    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    data = band.ReadAsArray(0, 0, cols, rows)

    points_list = [(u[0], u[1])] #list of X,Y coordinates

    for point in points_list:
        col = int((point[0] - xOrigin) / pixelWidth)
        row = int((yOrigin - point[1] ) / pixelHeight)
    if x_or_y == 'x':
      return col
    return row
  except:
    return -1

# df['TopLeft_x'] = df.apply(lambda x: get_utm(x.Left, x.Bottom, x.fileName, 'x'), axis=1)
# df['TopLeft_y'] = df.apply(lambda x: get_utm(x.Left, x.Bottom, x.fileName, 'y'), axis=1)

# df['BottomRight_x'] = df.apply(lambda x: get_utm(x.Right, x.Top, x.fileName, 'x'), axis=1)
# df['BottomRight_y'] = df.apply(lambda x: get_utm(x.Right, x.Top, x.fileName, 'y'), axis=1)

In [ ]:
df['TopLeft_x'] = 0
df['TopLeft_y'] = 0

df['BottomRight_x'] = 0
df['BottomRight_y'] = 0

In [ ]:
print(r.keys())

In [ ]:
# for d in r['data']['availableDownloads']:
#     remove = remove_download(d['downloadId'])

In [ ]:
df.reset_index(inplace=True)

In [ ]:
!rm -r data
!rm -r temp

In [ ]:
a = None
with open('data.json', 'r') as openfile:
    json_file = json.load(openfile)
    username, password = json_file['username'], json_file['password']
    login(username, password)
    a = download_retrieve()

In [ ]:
!rm -r data
!rm -r temp

In [ ]:
%%time
last_index = 0
# download_request_mapping = {}
# with open('data.json', 'r') as openfile:
#     json_file = json.load(openfile)
#     username, password = json_file['username'], json_file['password']
#     login(username, password)
#     dr = download_retrieve()
#     for d in dr['data']['available']:
#       if d['displayId'] not in download_request_mapping.keys():
#         download_request_mapping[d['displayId']] = d['url']

for i in tqdm(range(10, df.shape[0], 10), position=0, leave=True):
  e_ids, p_ids, filenames = df.iloc[last_index:i]['entity_id'].tolist(), df.iloc[last_index:i]['product_id'].tolist(), df.iloc[last_index:i]['fileName'].tolist()
  with open('data.json', 'r') as openfile:
    json_file = json.load(openfile)
    username, password = json_file['username'], json_file['password']
    login(username, password)
    # for idx, fn in enumerate(filenames):
    #   if fn not in download_request_mapping.keys():
    #     prev_len = len(download_retrieve()['data']['available'])
    #     r = bulk_download_scene([e_ids[idx]], [p_ids[idx]])
    #     while prev_len == len(download_retrieve()['data']['available']):
    #       pass
    #     download_request_mapping[fn] = download_retrieve()['data']['available'][0]['url']

    r = bulk_download_scene(e_ids, p_ids)

    download_urls = []
    for down in r['data']['availableDownloads']:
      download_urls.append(down['url'])

    for down in r['data']['preparingDownloads']:
      download_urls.append(down['url'])


    #print('Number downloads = ' + str(len(download_urls)))
    !mkdir temp
    def download(url):
      wget.download(url, '/content/temp')
    #print('Downloading images')
    res = ThreadPool(20)
    res.imap_unordered(download, download_urls)
    res.close()
    res.join()

    #print('Unzipping image files')
    !mkdir data
    def unzip(filename):
      with zipfile.ZipFile(filename, 'r') as f:
        f.extractall('./data/')

    zip_files = glob('./temp/*.ZIP')
    res = ThreadPool(20)
    res.imap_unordered(unzip, zip_files)
    res.close()
    res.join()

    #print('Getting UTM pixel coordinates')
    #print(df.iloc[last_index:i].head(10))
    df.iloc[last_index:i]['TopLeft_x'] = df.iloc[last_index:i].swifter.apply(lambda x: get_utm(x.Left, x.Bottom, x.fileName, 'x'), axis=1)
    df.iloc[last_index:i]['TopLeft_y'] = df.iloc[last_index:i].swifter.apply(lambda x: get_utm(x.Left, x.Bottom, x.fileName, 'y'), axis=1)

    df.iloc[last_index:i]['BottomRight_x'] = df.iloc[last_index:i].swifter.apply(lambda x: get_utm(x.Right, x.Top, x.fileName, 'x'), axis=1)
    df.iloc[last_index:i]['BottomRight_y'] = df.iloc[last_index:i].swifter.apply(lambda x: get_utm(x.Right, x.Top, x.fileName, 'y'), axis=1)

    to_be_moved = []
    downloaded_files = [os.path.basename(x)[0:-4] for x in glob('/content/bucket_data/*')]
    for f in glob('/content/data/*'):
      if f.split('/')[-1] not in downloaded_files:
        to_be_moved.append(f)

    def move_data(filename):
      new_path = '/content/bucket_data/' + filename.split('/')[-1]
      shutil.copy(filename, new_path)
    res = ThreadPool(20)
    res.imap_unordered(move_data, to_be_moved)
    res.close()
    res.join()

    !rm -r temp
    !rm -r data
    last_index = i

In [ ]:
df.to_csv('./GeoData_largest.csv')
df.to_csv('/content/drive/MyDrive/GeoData_largest.csv')

In [ ]:
df.head()

In [ ]:
from d2l import torch as d2l

def bbox_to_rect(bbox, color):
    """Convert bounding box to matplotlib format."""
    # Convert the bounding box (upper-left x, upper-left y, lower-right x,
    # lower-right y) format to the matplotlib format: ((upper-left x,
    # upper-left y), width, height)
    return d2l.plt.Rectangle(
        xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0], height=bbox[3]-bbox[1],
        fill=False, edgecolor=color, linewidth=2)

In [ ]:
def get_bbox(pixel_coords):
  return [pixel_coords[2],pixel_coords[3],pixel_coords[0],pixel_coords[1]]

df['bbox'] = df[['TopLeft_x', 'TopLeft_y', 'BottomRight_x', 'BottomRight_y']].apply(lambda x:
                                                                                    get_bbox([x.TopLeft_x, x.TopLeft_y, x.BottomRight_x, x.BottomRight_y]),axis=1)

In [ ]:
def combine_bboxes(bboxes):
  result = []
  for b in bboxes:
    result.append(b)
  return result

def combine_labels(labels):
  result = []
  for l in labels:
    result.append(l)
  return result

data_df = df.groupby(['fileName']).agg({'bbox':lambda x: combine_bboxes(x), 'Class':lambda x: combine_labels(x),
                                        'entity_id':'min', 'product_id':'min'}).reset_index()

In [ ]:
data_df.to_csv('/content/drive/MyDrive/ATML_Dataset_Largest.csv')

In [ ]:
df[df.TopLeft_x == 0].shape

In [ ]:
data_df.head()

In [ ]:
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import matplotlib.patches as patches

temp = df.iloc[107]

path = '/content/bucket_data/' + temp.fileName.lower() + '.tif'
print(path)
with Image.open(path) as image:
    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)
    print(image.size)
    # Create a Rectangle patch

    # Add the patch to the Axes
    for index, row in df[df.fileName == temp.fileName].iterrows():
      ax.add_patch(bbox_to_rect([row.TopLeft_x, row.TopLeft_y, row.BottomRight_x, row.BottomRight_y], 'red'))
      ax.annotate(row.Class, ((row.TopLeft_x + row.BottomRight_x)/2.2,
                              row.BottomRight_y-100))

    plt.show()


Build dataset for pytorch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class VHRSatelliteDataset(Dataset):
  def __init__(self, fileNames, boxes, labels, entities, products, transforms=None):
    self.files = fileNames
    self.boxes = boxes
    self.labels = labels
    self.entities = entities
    self.products = products
    self.transforms = transforms
  
  def __len__(self):
    return len(self.files)

  def __getitem__(self, idx):
    file_path = self.files[idx]
    label = self.labels[idx]
    boxes = self.boxes[idx]

    e_ids, p_ids = [self.entities[idx]], [self.products[idx]]
    r = None
    with open('data.json', 'r') as openfile:
      json_file = json.load(openfile)
      username, password = json_file['username'], json_file['password']
      login(username, password)
      r = bulk_download_scene(e_ids, p_ids)
      while len(r['data']['preparingDownloads']) > 0:
        r = bulk_download_scene(e_ids, p_ids)

    !mkdir temp
    def download(url):
      wget.download(url['url'], '/content/temp')

    resDownload = ThreadPool(20)
    resDownload.imap_unordered(download, r['data']['availableDownloads'])
    resDownload.close()
    resDownload.join()

    !mkdir data
    def unzip(filename):
      with zipfile.ZipFile(filename, 'r') as f:
        f.extractall('./data/')

    zip_files = glob('./temp/*.ZIP')
    res = ThreadPool(20)
    res.imap_unordered(unzip, zip_files)
    res.close()
    res.join()

    path = '/content/data/' + file_path.lower() + '.tif'

    img = Image.open(path)

    return img, label, boxes

In [ ]:
data = VHRSatelliteDataset(data_df['fileName'].tolist(), data_df['bbox'].tolist(), data_df['Class'].tolist(),
                           data_df['entity_id'].tolist(),data_df['product_id'].tolist())

In [ ]:
a = DataLoader(data, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir googleBucketFolder
!gcsfuse --implicit-dirs colab-connect-bucket googleBucketFolder

In [ ]:
!mkdir bucket_data
!gcsfuse --implicit-dirs atml_bucket bucket_data

In [ ]:
%%time
for _, data in a:
  pass
  !rm -r data
  !rm -r temp